<a href="https://colab.research.google.com/github/KalanaBimsara/Clone_Me/blob/main/clone_me.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install transformers datasets accelerate bitsandbytes -q
!pip install peft -q  # LoRA fine-tuning
!pip install torch torchvision torchaudio -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import os

# Path where JSON files are stored
chat_folder = "/content"  # Adjust if uploaded elsewhere
chat_data = []

for filename in os.listdir(chat_folder):
    if filename.endswith(".json"):
        with open(os.path.join(chat_folder, filename), "r", encoding="utf-8") as file:
            data = json.load(file)
            chat_data.extend(data.get("messages", []))

# Extract messages only
messages = []
for msg in chat_data:
    if "content" in msg:
        messages.append(msg["content"])

# Save processed messages for training
with open("chat_data.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(messages))

print(f"Processed {len(messages)} messages for training.")


Processed 32318 messages for training.


In [ ]:

!pip install datasets

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Add padding token (GPT-2 does not have one by default)
tokenizer.pad_token = tokenizer.eos_token

# Load dataset
dataset = load_dataset("text", data_files={"train": "chat_data.txt"}, split="train")

# Tokenize dataset properly (FIX: Include `labels`)
def tokenize_function(examples):
    tokens = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=1024)
    return {
        "input_ids": tokens["input_ids"],
        "attention_mask": tokens["attention_mask"],
        "labels": tokens["input_ids"]  # ✅ FIX: Labels must be the same as `input_ids` for causal models
    }

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Set PyTorch format
tokenized_datasets.set_format("torch")

# Split dataset (90% train, 10% eval)
split_dataset = tokenized_datasets.train_test_split(test_size=0.1)

# Assign datasets
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print("Dataset tokenized successfully!")


Dataset tokenized successfully!


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

# Load GPT-2 model
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2_finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    eval_strategy="no",
    push_to_hub=False
)

# Ensure dataset contains `labels`
if "labels" not in train_dataset.column_names:
    raise ValueError("The dataset is missing `labels`! Check tokenization.")

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Start training
trainer.train()

# Save fine-tuned model
model.save_pretrained("gpt2_finetuned")
tokenizer.save_pretrained("gpt2_finetuned")

print("Training complete!")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,0.365600
200,0.027500
300,0.022400
400,0.024700
500,0.022800
600,0.023500
700,0.023100
800,0.019500
900,0.021900
1000,0.022100


Step,Training Loss
100,0.365600
200,0.027500
300,0.022400
400,0.024700
500,0.022800
600,0.023500
700,0.023100
800,0.019500
900,0.021900
1000,0.022100
